In [1]:
import pandas as pd
import random
import numpy as np
import matplotlib.pyplot as plt

from sklearn.preprocessing import MinMaxScaler
from sklearn.neighbors import NearestNeighbors
from sklearn.metrics import silhouette_score

from datetime import datetime, timedelta

from statistics import mean

from sklearn.decomposition import PCA

import requests
import pandas as pd
from requests_html import HTML
from requests_html import HTMLSession

from bs4 import BeautifulSoup

In [2]:
random.seed(3888)

In [3]:
df = pd.read_csv("data/data.txt")
df = df.drop(columns='Unnamed: 0')

df = df.rename(columns = {'tourist_service_index': 'tourist_service_infrastructure'})

# An Australian wouldn't travel to Australia!
df = df[df["iso_code"] != "AUS"]

In [4]:
df_without_covid = df.drop(columns=['new_cases_per_million', 
                                    'new_cases_smoothed_per_million', 
                                    'stringency_index', 
                                    'positive_rate', 
                                    'human_development_index', 
                                    'international_travel_controls',
                                    'cost_living_index',
                                    'date'])
non_covid_colnames = ['iso_code', 'location', 'continent', 'tourist_service_infrastructure', 'art_gallery']
covid_colnames = ['iso_code', 'new_cases_smoothed_per_million', 'date']
df_without_covid = df_without_covid.drop_duplicates()
df_without_covid = df_without_covid.reset_index()
df_without_covid = df_without_covid.drop(columns=['index'])

df_without_covid

,iso_code,location,continent,tourist_service_infrastructure
0,ALB,Albania,Europe,4.01
1,DZA,Algeria,Africa,1.84
2,ARG,Argentina,South America,4.53
3,ARM,Armenia,Asia,4.33
4,AUT,Austria,Europe,6.66
...,...,...,...,...
110,VEN,Venezuela,South America,2.96
111,VNM,Vietnam,Asia,2.85
112,YEM,Yemen,Asia,1.87
113,ZMB,Zambia,Africa,2.52


In [5]:
full_tourism = pd.read_csv("data/full_tourism.csv")
full_tourism = full_tourism[full_tourism["Country ISO3"] != "AUS"]
indicators = {
                'WEF Infrastructure subindex, 1-7 (best)': 'infrastructure', 
                'WEF Natural and cultural resources subindex, 1-7 (best)': 'natural_cultural_resources',
                'WEF Safety and security pillar, 1-7 (best)': 'safety_security',
                'WEF Health and hygiene, 1-7 (best)': 'health_hygiene',
                'WEF Price competitiveness in the Travel and Tourism Industry pillar, 1-7 (best)': 'price_competitiveness',
                'WEF Air transport infrastructure, 1-7 (best)': 'air_transport',
                'WEF Ground and port infrastructure, 1-7 (best)': 'ground_port'
}

full_tourism_req_indicators = full_tourism[full_tourism["Indicator"].isin(indicators)]
full_tourism_req_indicators = full_tourism_req_indicators[['Country ISO3', 'Indicator', 'Subindicator Type', '2019']]
full_tourism_req_indicators = full_tourism_req_indicators[full_tourism["Subindicator Type"] == "Value"]
full_tourism_req_indicators = full_tourism_req_indicators.drop(columns = ['Subindicator Type'])
full_tourism_req_indicators = full_tourism_req_indicators.rename(columns = {'Country ISO3': 'iso_code'})
full_tourism_req_indicators = full_tourism_req_indicators.set_index('iso_code')

inds = pd.DataFrame()

for ind in indicators.keys():
    inds[indicators[ind]] = full_tourism_req_indicators[full_tourism_req_indicators["Indicator"] == ind].drop(columns = ["Indicator"]).rename(columns = {'2019': indicators[ind]})[indicators[ind]]
    
non_covid = pd.merge(inds, df_without_covid, on='iso_code')

<ipython-input-5-9784066d7299>:15: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  full_tourism_req_indicators = full_tourism_req_indicators[full_tourism["Subindicator Type"] == "Value"]


In [6]:
covid = pd.read_csv("https://raw.githubusercontent.com/owid/covid-19-data/master/public/data/owid-covid-data.csv")

covid = covid[covid["iso_code"] != "AUS"]
covid_needed_cols = covid[covid_colnames]
covid_needed_cols['date'] = pd.to_datetime(covid_needed_cols['date'], format='%Y-%m-%d')

<ipython-input-6-a0bce08db393>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  covid_needed_cols['date'] = pd.to_datetime(covid_needed_cols['date'], format='%Y-%m-%d')


In [7]:
covid_needed_cols = covid_needed_cols[covid_needed_cols['date'] >= datetime.now() - timedelta(days = 30)]
poi = pd.read_json("data/poi_types.json")
poi = poi.fillna(0)
poi = poi.replace(0, np.nan)
poi = poi.dropna(how='all', axis=0)
poi = poi.replace(np.nan, 0)
poi = poi.transpose()
poi = poi[poi.index != "Australia"]
iso_location = df[["iso_code", "location"]].drop_duplicates()

iso_location.to_csv("iso_location.csv")

In [8]:
def iso_code_to_loc(iso_code):
    return iso_location[iso_location["iso_code"] == iso_code]["location"].iloc[0]
def loc_to_iso_code(loc):
    return iso_location[iso_location["location"] == loc]["iso_code"].iloc[0]
poi = poi.set_index(loc_to_iso_code(loc) for loc in poi.index)
poi.columns

Index(['art_gallery', 'food', 'museum', 'park', 'restaurant', 'church',
       'mosque', 'place_of_worship', 'zoo', 'travel_agency', 'amusement_park',
       'aquarium', 'cafe', 'store', 'cemetery', 'hindu_temple',
       'natural_feature', 'library', 'campground', 'lodging', 'casino',
       'local_government_office', 'liquor_store', 'bar', 'shopping_mall',
       'spa', 'transit_station', 'grocery_or_supermarket', 'synagogue',
       'movie_theater', 'general_contractor', 'parking', 'book_store',
       'night_club', 'city_hall', 'clothing_store', 'department_store',
       'health', 'hospital'],
      dtype='object')

In [9]:
# removing less relevant columns

poi = poi.drop(columns=['travel_agency',
                        'store',
                        'cemetery',
                        'library',
                        'campground',
                        'lodging',
                        'local_government_office',
                        'liquor_store',
                        'transit_station',
                        'grocery_or_supermarket',
                        'movie_theater',
                        'general_contractor',
                        'parking',
                        'book_store',
                        'city_hall',
                        'health',
                        'hospital'
                       ])

In [10]:
poi['iso_code'] = poi.index
covid_merged = pd.merge(covid_needed_cols, non_covid, on="iso_code")
covid_merged = pd.merge(covid_merged, poi, on="iso_code")

In [11]:
# https://practicaldatascience.co.uk/data-science/how-to-read-an-rss-feed-in-python

def get_source(url):
    """Return the source code for the provided URL. 

    Args: 
        url (string): URL of the page to scrape.

    Returns:
        response (object): HTTP response object from requests_html. 
    """

    try:
        session = HTMLSession()
        response = session.get(url)
        return response

    except requests.exceptions.RequestException as e:
        print(e)


def get_feed(url):
    """Return a Pandas dataframe containing the RSS feed contents.

    Args: 
        url (string): URL of the RSS feed to read.

    Returns:
        df (dataframe): Pandas dataframe containing the RSS feed contents.
    """
    
    response = get_source(url)
    
    df = pd.DataFrame(columns = ['title', 'pubDate', 'guid', 'description'])

    with response as r:
        items = r.html.find("item", first=False)

        for item in items:        

            title = item.find('title', first=True).text
            pubDate = item.find('pubDate', first=True).text
            guid = item.find('guid', first=True).text
            description = item.find('description', first=True).text

            row = {'title': [title], 'pubDate': [pubDate], 'guid': [guid], 'description': [description]}
            df = pd.concat([df, pd.DataFrame.from_dict(row)])

    return df

In [12]:
travel_advice = get_feed("https://www.smartraveller.gov.au/countries/documents/index.rss")
travel_advice = travel_advice[travel_advice["title"] != "No travel advice"]
travel_advice = travel_advice.drop(columns=['guid'])
travel_advice

,title,pubDate,description
0,Mali,07 May 2022 22:00:00 AEST,"On 6 May 2022, the US government issued an ale..."
0,Sri Lanka,07 May 2022 22:00:00 AEST,A Public Emergency has been declared in Sri La...
0,Timor-Leste,06 May 2022 22:00:00 AEST,The Presidential inauguration will be held on ...
0,Papua New Guinea,06 May 2022 22:00:00 AEST,"If you&#039;re not a PNG citizen, you&#039;ll ..."
0,Hong Kong,06 May 2022 22:00:00 AEST,Fully vaccinated travellers can enter Hong Kon...
...,...,...,...
0,Costa Rica,28 Oct 2021 23:00:00 AEDT,If you&#039;re not fully vaccinated against CO...
0,Marshall Islands,28 Oct 2021 23:00:00 AEDT,"Due to COVID-19, Marshall Islands has closed i..."
0,Niger,28 Oct 2021 23:00:00 AEDT,The US Government has issued a security alert ...
0,North Korea (Democratic People's Republic of K...,28 Oct 2021 23:00:00 AEDT,North Korea’s borders remain closed due to COV...


In [13]:
replacements = {
    "United States of America": "United States",
    "Israel and the Palestinian Territories": "Israel",
    "South Korea (Republic of Korea)": "South Korea"
}
for replacement in replacements:
    travel_advice.replace(replacement, replacements[replacement], inplace = True)
set(covid_merged["location"]).difference(set(travel_advice["title"]))

{'Barbados', 'Luxembourg', 'Suriname'}

In [14]:
travel_advice.rename(columns={"title": "location", "description": "advice"}, inplace = True)
travel_advice["advice"] = [BeautifulSoup(s, "lxml").text for s in travel_advice["advice"]]
covid_merged = pd.merge(covid_merged, travel_advice, on="location")
covid_merged

,iso_code,new_cases_smoothed_per_million,date,infrastructure,natural_cultural_resources,safety_security,health_hygiene,price_competitiveness,air_transport,ground_port,...,casino,bar,shopping_mall,spa,synagogue,night_club,clothing_store,department_store,pubDate,advice
0,ALB,17.354,2022-04-10,3.08,2.04,5.77,5.28,5.26,2.10,3.11,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,13 Apr 2022 22:00:00 AEST,Face coverings must be worn at all times in in...
1,ALB,15.216,2022-04-11,3.08,2.04,5.77,5.28,5.26,2.10,3.11,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,13 Apr 2022 22:00:00 AEST,Face coverings must be worn at all times in in...
2,ALB,13.525,2022-04-12,3.08,2.04,5.77,5.28,5.26,2.10,3.11,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,13 Apr 2022 22:00:00 AEST,Face coverings must be worn at all times in in...
3,ALB,13.177,2022-04-13,3.08,2.04,5.77,5.28,5.26,2.10,3.11,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,13 Apr 2022 22:00:00 AEST,Face coverings must be worn at all times in in...
4,ALB,13.326,2022-04-14,3.08,2.04,5.77,5.28,5.26,2.10,3.11,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,13 Apr 2022 22:00:00 AEST,Face coverings must be worn at all times in in...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3136,ZWE,3.057,2022-05-03,2.34,2.44,5.39,2.96,5.32,1.79,2.27,...,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,11 Apr 2022 22:00:00 AEST,We now advise you exercise a high degree of ca...
3137,ZWE,3.038,2022-05-04,2.34,2.44,5.39,2.96,5.32,1.79,2.27,...,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,11 Apr 2022 22:00:00 AEST,We now advise you exercise a high degree of ca...
3138,ZWE,2.518,2022-05-05,2.34,2.44,5.39,2.96,5.32,1.79,2.27,...,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,11 Apr 2022 22:00:00 AEST,We now advise you exercise a high degree of ca...
3139,ZWE,3.521,2022-05-06,2.34,2.44,5.39,2.96,5.32,1.79,2.27,...,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,11 Apr 2022 22:00:00 AEST,We now advise you exercise a high degree of ca...


In [15]:
covid_colnames.append("advice")
covid_merged = covid_merged[list(set(non_covid_colnames).union(set(covid_colnames)))]
covid_colnames.remove("advice")
covid_merged

,date,art_gallery,location,iso_code,tourist_service_infrastructure,advice,continent,new_cases_smoothed_per_million
0,2022-04-10,3.0,Albania,ALB,4.01,Face coverings must be worn at all times in in...,Europe,17.354
1,2022-04-11,3.0,Albania,ALB,4.01,Face coverings must be worn at all times in in...,Europe,15.216
2,2022-04-12,3.0,Albania,ALB,4.01,Face coverings must be worn at all times in in...,Europe,13.525
3,2022-04-13,3.0,Albania,ALB,4.01,Face coverings must be worn at all times in in...,Europe,13.177
4,2022-04-14,3.0,Albania,ALB,4.01,Face coverings must be worn at all times in in...,Europe,13.326
...,...,...,...,...,...,...,...,...
3136,2022-05-03,0.0,Zimbabwe,ZWE,2.95,We now advise you exercise a high degree of ca...,Africa,3.057
3137,2022-05-04,0.0,Zimbabwe,ZWE,2.95,We now advise you exercise a high degree of ca...,Africa,3.038
3138,2022-05-05,0.0,Zimbabwe,ZWE,2.95,We now advise you exercise a high degree of ca...,Africa,2.518
3139,2022-05-06,0.0,Zimbabwe,ZWE,2.95,We now advise you exercise a high degree of ca...,Africa,3.521


In [17]:
descriptions = pd.read_csv("data/country_descriptions_cleaned_2.csv")
covid_merged = pd.merge(descriptions, covid_merged, on="iso_code")
covid_merged

,iso_code,description,date,art_gallery,location,tourist_service_infrastructure,advice,continent,new_cases_smoothed_per_million
0,ALB,Albania (Albanian: Shqipëria) is a country in ...,2022-04-10,3.0,Albania,4.01,Face coverings must be worn at all times in in...,Europe,17.354
1,ALB,Albania (Albanian: Shqipëria) is a country in ...,2022-04-11,3.0,Albania,4.01,Face coverings must be worn at all times in in...,Europe,15.216
2,ALB,Albania (Albanian: Shqipëria) is a country in ...,2022-04-12,3.0,Albania,4.01,Face coverings must be worn at all times in in...,Europe,13.525
3,ALB,Albania (Albanian: Shqipëria) is a country in ...,2022-04-13,3.0,Albania,4.01,Face coverings must be worn at all times in in...,Europe,13.177
4,ALB,Albania (Albanian: Shqipëria) is a country in ...,2022-04-14,3.0,Albania,4.01,Face coverings must be worn at all times in in...,Europe,13.326
...,...,...,...,...,...,...,...,...,...
3080,GBR,The United Kingdom of Great Britain and Northe...,2022-05-03,6.0,United Kingdom,6.10,All COVID-19 travel restrictions in the UK hav...,Europe,166.836
3081,GBR,The United Kingdom of Great Britain and Northe...,2022-05-04,6.0,United Kingdom,6.10,All COVID-19 travel restrictions in the UK hav...,Europe,165.496
3082,GBR,The United Kingdom of Great Britain and Northe...,2022-05-05,6.0,United Kingdom,6.10,All COVID-19 travel restrictions in the UK hav...,Europe,162.132
3083,GBR,The United Kingdom of Great Britain and Northe...,2022-05-06,6.0,United Kingdom,6.10,All COVID-19 travel restrictions in the UK hav...,Europe,159.615


## Quantitative Analysis

In [18]:
covid_merged_no_quant = list(set(covid_merged.columns).difference(set(covid_merged.select_dtypes(include=[np.number]).columns)))
covid_merged_no_quant.remove("date")
medians = covid_merged.groupby(["iso_code"]).median()
medians = medians.fillna(covid_merged.median())
medians

<ipython-input-18-9d229b89208b>:4: FutureWarning: DataFrame.mean and DataFrame.median with numeric_only=None will include datetime64 and datetime64tz columns in a future version.
  medians = medians.fillna(covid_merged.median())


,art_gallery,tourist_service_infrastructure,new_cases_smoothed_per_million
iso_code,,,
ALB,3.0,4.01,15.0915
ARE,0.0,5.63,23.8930
ARG,0.0,4.53,32.2895
ARM,0.0,4.33,3.3210
AUT,0.0,6.66,828.4320
...,...,...,...
VNM,0.0,2.85,194.8225
YEM,0.0,1.87,0.0050
ZAF,0.0,4.30,48.7420


In [19]:
all_data_w_medians = pd.merge(medians, covid_merged[covid_merged_no_quant], on="iso_code").drop_duplicates()
all_data_w_medians = all_data_w_medians.set_index(all_data_w_medians["iso_code"]).drop(columns=["iso_code"])
all_data_w_medians

,art_gallery,tourist_service_infrastructure,new_cases_smoothed_per_million,location,description,advice,continent
iso_code,,,,,,,
ALB,3.0,4.01,15.0915,Albania,Albania (Albanian: Shqipëria) is a country in ...,Face coverings must be worn at all times in in...,Europe
ARE,0.0,5.63,23.8930,United Arab Emirates,The United Arab Emirates (Arabic: دولة الإمارا...,You no longer need a COVID-19 test to travel t...,Asia
ARG,0.0,4.53,32.2895,Argentina,Argentina is a large country in the southern p...,You no longer need to have a COVID-19 test or ...,South America
ARM,0.0,4.33,3.3210,Armenia,Armenia (Armenian: Հայաստան Hayastan) is a lan...,You can enter Armenia with a negative COVID-19...,Asia
AUT,0.0,6.66,828.4320,Austria,"Austria (German: Österreich, literally ""the Ea...",Most COVID-19 measures across Austria have bee...,Europe
...,...,...,...,...,...,...,...
VNM,0.0,2.85,194.8225,Vietnam,Vietnam (Vietnamese: Việt Nam) is a country in...,We now advise you exercise a high degree of ca...,Asia
YEM,0.0,1.87,0.0050,Yemen,Yemen (Arabic: ٱلْيَمَن) is a country in the M...,We've reviewed our travel advice for Yemen. We...,Asia
ZAF,0.0,4.30,48.7420,South Africa,"SA redirects here. For the state, see South Au...",Heavy rains and flooding in the city of Durban...,Africa


### Scaling

In [20]:
iso_code = medians.index
scaler = MinMaxScaler()
medians_scaled = scaler.fit_transform(medians)
cols = list(set(non_covid_colnames).union(set(covid_colnames)))
to_remove = ['iso_code', 'continent', 'location', 'date']

for col in to_remove:
    cols.remove(col)
    
medians_scaled = pd.DataFrame(medians_scaled, 
                              columns = cols, 
                              index = iso_code)
medians_scaled

,tourist_service_infrastructure,art_gallery,new_cases_smoothed_per_million
iso_code,,,
ALB,0.5,0.458753,0.009300
ARE,0.0,0.784708,0.014725
ARG,0.0,0.563380,0.019899
ARM,0.0,0.523139,0.002047
AUT,0.0,0.991952,0.510540
...,...,...,...
VNM,0.0,0.225352,0.120064
YEM,0.0,0.028169,0.000003
ZAF,0.0,0.517103,0.030038


In [21]:
# perform PCA if > 2 attributes selected

if len(medians_scaled.columns) > 2:
    pca = PCA(n_components=2)
    pc = pca.fit_transform(medians_scaled)
    medians_scaled = pd.DataFrame(data = pc, columns = ['PC1', 'PC2'], index = medians_scaled.index)
medians_scaled

,PC1,PC2
iso_code,,
ALB,-0.072664,0.433681
ARE,0.153815,-0.071691
ARG,-0.024459,-0.066682
ARM,-0.067538,-0.063599
AUT,0.604888,-0.134246
...,...,...
VNM,-0.244370,-0.069695
YEM,-0.473951,-0.050823
ZAF,-0.056594,-0.066682


### Finding 5 Neighbours

In [22]:
num_neighbours = 5
dist_metrics = ['euclidean', 'manhattan', 'chebyshev', 'cosine', 'cityblock', 'braycurtis', 'canberra',
               'correlation', 'minkowski']
location_neighbours = {}

for metric in dist_metrics:
    nbrs = NearestNeighbors(metric = metric, 
                            n_neighbors = num_neighbours + 1, 
                            algorithm='auto').fit(medians_scaled)
    
    nbr_indices = list(list(x) for x in nbrs.kneighbors(medians_scaled)[1])
    
    iso_location = df[["iso_code", "location"]].drop_duplicates()

    for i in range(len(nbr_indices)):
        current_iso_code = list(medians_scaled.index)[i]
        current_location = iso_code_to_loc(current_iso_code)

        neighbours = []
        for j in range(1, num_neighbours + 1):
            iso_code = medians_scaled.index[nbr_indices[i][j]]
            neighbours.append(iso_code_to_loc(iso_code))

        if not current_location in location_neighbours:
            location_neighbours[current_location] = {metric: neighbours}
        else:
            location_neighbours[current_location][metric] = neighbours
location_neighbours_df = pd.DataFrame(location_neighbours).transpose()
location_neighbours_df

,euclidean,manhattan,chebyshev,cosine,cityblock,braycurtis,canberra,correlation,minkowski
Albania,"[Serbia, Morocco, Colombia, Russia, Kazakhstan]","[Morocco, Serbia, Colombia, Russia, Kazakhstan]","[Serbia, Morocco, Colombia, Russia, Kazakhstan]","[Serbia, Morocco, Colombia, Russia, Kazakhstan]","[Morocco, Serbia, Colombia, Russia, Kazakhstan]","[Morocco, Serbia, Colombia, Russia, Kazakhstan]","[Morocco, Serbia, Colombia, Russia, Kazakhstan]","[Nicaragua, North Macedonia, Morocco, Nigeria,...","[Serbia, Morocco, Colombia, Russia, Kazakhstan]"
United Arab Emirates,"[Montenegro, Costa Rica, Mauritius, Denmark, P...","[Montenegro, Costa Rica, Mauritius, Peru, Denm...","[Montenegro, Costa Rica, Mauritius, Denmark, P...","[Montenegro, Finland, Estonia, Belgium, Israel]","[Montenegro, Costa Rica, Mauritius, Peru, Denm...","[Montenegro, Costa Rica, Mauritius, Peru, Denm...","[Montenegro, Costa Rica, Mauritius, Peru, Bulg...","[Romania, South Korea, Costa Rica, Jamaica, Do...","[Montenegro, Costa Rica, Mauritius, Denmark, P..."
Argentina,"[Brazil, Poland, Chile, Romania, Myanmar]","[Brazil, Poland, Chile, Romania, Myanmar]","[Brazil, Poland, Chile, Romania, Myanmar]","[Chile, Brazil, Poland, Romania, Myanmar]","[Brazil, Poland, Chile, Romania, Myanmar]","[Brazil, Poland, Chile, Romania, South Africa]","[Brazil, Poland, Chile, Romania, Myanmar]","[Costa Rica, Dominican Republic, United Arab E...","[Brazil, Poland, Chile, Romania, Myanmar]"
Armenia,"[Ukraine, South Africa, Myanmar, Tunisia, Oman]","[Ukraine, South Africa, Myanmar, Tunisia, Oman]","[Ukraine, South Africa, Myanmar, Tunisia, Oman]","[Ukraine, South Africa, Myanmar, Tunisia, Oman]","[Ukraine, South Africa, Myanmar, Tunisia, Oman]","[Ukraine, South Africa, Myanmar, Tunisia, Oman]","[Ukraine, South Africa, Myanmar, Tunisia, Oman]","[Netherlands, Mongolia, Moldova, Nicaragua, Ne...","[Ukraine, South Africa, Myanmar, Tunisia, Oman]"
Austria,"[Portugal, Germany, Cyprus, Italy, South Korea]","[Portugal, Cyprus, Germany, Italy, South Korea]","[Germany, Portugal, Cyprus, Italy, South Korea]","[Portugal, Spain, United States, New Zealand, ...","[Portugal, Cyprus, Germany, Italy, South Korea]","[Portugal, Cyprus, Germany, Italy, South Korea]","[Portugal, Italy, Germany, Cyprus, South Korea]","[Romania, Italy, Japan, Iceland, Jamaica]","[Portugal, Germany, Cyprus, Italy, South Korea]"
...,...,...,...,...,...,...,...,...,...
Vietnam,"[Sri Lanka, Paraguay, Cambodia, Egypt, Bolivia]","[Sri Lanka, Paraguay, Cambodia, Egypt, Bolivia]","[Paraguay, Sri Lanka, Egypt, Cambodia, Azerbai...","[Honduras, Nicaragua, China, Bolivia, Philippi...","[Sri Lanka, Paraguay, Cambodia, Egypt, Bolivia]","[Sri Lanka, Paraguay, Cambodia, Egypt, Azerbai...","[Sri Lanka, Paraguay, Cambodia, Egypt, Bolivia]","[Sri Lanka, Jordan, Lebanon, India, Kazakhstan]","[Sri Lanka, Paraguay, Cambodia, Egypt, Bolivia]"
Yemen,"[Algeria, Bangladesh, Ethiopia, Cameroon, Nepal]","[Algeria, Bangladesh, Ethiopia, Cameroon, Nepal]","[Algeria, Bangladesh, Ethiopia, Cameroon, Nepal]","[Algeria, Bangladesh, Ethiopia, Cameroon, Nepal]","[Algeria, Bangladesh, Ethiopia, Cameroon, Nepal]","[Algeria, Bangladesh, Ethiopia, Cameroon, Nepal]","[Algeria, Bangladesh, Ethiopia, Cameroon, Nepal]","[Nicaragua, North Macedonia, Morocco, Nigeria,...","[Algeria, Bangladesh, Ethiopia, Cameroon, Nepal]"
South Africa,"[Myanmar, Armenia, Ukraine, Poland, Chile]","[Myanmar, Armenia, Ukraine, Poland, Argentina]","[Myanmar, Armenia, Ukraine, Poland, Chile]","[Myanmar, Armenia, Ukraine, Poland, Tunisia]","[Myanmar, Armenia, Ukraine, Poland, Argentina]","[Myanmar, Armenia, Ukraine, Poland, Chile]","[Myanmar, Armenia, Ukraine, Tunisia, Poland]","[Romania, Italy, Japan, Iceland, Jamaica]","[Myanmar, Armenia, Ukraine, Poland, Chile]"
Zambia,"[Tanzania, Pakistan, Ghana, Uganda, India]","[Tanzania, Pakistan, Ghana, Uganda, India]","[Tanzania, Pakistan, Ghana, Uganda, India]","[Tanzania, Pakistan, Ghana, Uganda, Nepal]","[Tanzania, Pakistan, Ghana, Uganda, India]"

In [23]:
prop_sim = []

for location in location_neighbours.keys():
    ind_pairs = []
    for i in range(0,len(location_neighbours[location])):
        for j in range(1,len(location_neighbours[location])):
            index = [i,j]
            s = sorted(index)
            if s not in ind_pairs and i != j:
                ind_pairs.append(index)
    for pair in ind_pairs:
        loc_intersect = set(location_neighbours[location][dist_metrics[pair[0]]]).intersection(location_neighbours[location][dist_metrics[pair[1]]])
        prop_sim.append(len(loc_intersect)/num_neighbours)
        
mean(prop_sim)

0.7135858585858585

In [24]:
def find_top_neighbours(country):
    d = {}
    lists = location_neighbours_df.loc[country].tolist()
    for ls in lists:
        for c in ls:
            if c in d:
                d[c] += 1
            else:
                d[c] = 1
    top = []

    for k,v in sorted(d.items(), key=lambda p:p[1], reverse=True)[:num_neighbours]:
        top.append(k)
    return top

In [25]:
locations = location_neighbours_df.index.tolist()
top_neighbours = {}
for location in locations:
    top_neighbours[location] = find_top_neighbours(location)
top_neighbours_df = pd.DataFrame(top_neighbours).transpose()
top_neighbours_df

,0,1,2,3,4
Albania,Morocco,Serbia,Colombia,Russia,Kazakhstan
United Arab Emirates,Montenegro,Costa Rica,Mauritius,Peru,Denmark
Argentina,Brazil,Poland,Chile,Romania,Myanmar
Armenia,Ukraine,South Africa,Myanmar,Tunisia,Oman
Austria,Portugal,Germany,Italy,Cyprus,South Korea
...,...,...,...,...,...
Vietnam,Sri Lanka,Paraguay,Cambodia,Egypt,Bolivia
Yemen,Algeria,Bangladesh,Ethiopia,Cameroon,Nepal
South Africa,Myanmar,Armenia,Ukraine,Poland,Chile
Zambia,Tanzania,Pakistan,Ghana,Uganda,India


In [26]:
top_neighbours_df.loc["United Kingdom"]

0        Croatia
1        Ireland
2       Slovenia
3    Netherlands
4       Slovakia
Name: United Kingdom, dtype: object

### Neighbours Evaluation

In [62]:
def evals(location_neighbours, n):
    prop_sim = []
    for location in location_neighbours.keys():
        ind_pairs = []
        for i in range(0,len(location_neighbours[location])):
            for j in range(1,len(location_neighbours[location])):
                index = [i,j]
                s = sorted(index)
                if s not in ind_pairs and i != j:
                    ind_pairs.append(index)
        for pair in ind_pairs:
            loc_intersect = set(location_neighbours[location][dist_metrics[n][pair[0]]]).intersection(location_neighbours[location][dist_metrics[n][pair[1]]])
            prop_sim.append(len(loc_intersect)/num_neighbours)
    return mean(prop_sim)

In [70]:
num_neighbours = 5
dist_metrics = []
eval_metric = {}
n = {}

m = ['euclidean', 'manhattan', 'chebyshev', 'cosine', 'cityblock', 'braycurtis', 'canberra',
               'correlation', 'minkowski']

for i in range(0, 9):
    temp = ['euclidean', 'manhattan', 'chebyshev', 'cosine', 'cityblock', 'braycurtis', 'canberra',
               'correlation', 'minkowski']
    temp.remove(temp[i])
    dist_metrics.append(temp)
    
for d in range(0, len(dist_metrics)):
    location_neighbours = {}
    for metric in dist_metrics[d]:
        nbrs = NearestNeighbors(metric = metric, 
                                n_neighbors = num_neighbours + 1, 
                                algorithm='auto').fit(medians_scaled)

        nbr_indices = list(list(x) for x in nbrs.kneighbors(medians_scaled)[1])

        iso_location = df[["iso_code", "location"]].drop_duplicates()

        for i in range(len(nbr_indices)):
            current_iso_code = list(medians_scaled.index)[i]
            current_location = iso_code_to_loc(current_iso_code)

            neighbours = []
            for j in range(1, num_neighbours + 1):
                iso_code = medians_scaled.index[nbr_indices[i][j]]
                neighbours.append(iso_code_to_loc(iso_code))

            if not current_location in location_neighbours:
                location_neighbours[current_location] = {metric: neighbours}
            else:
                location_neighbours[current_location][metric] = neighbours
    
    location_neighbours_df = pd.DataFrame(location_neighbours).transpose()
    eval_metric[m[d]] = evals(location_neighbours, d)

    locations = location_neighbours_df.index.tolist()
    top_neighbours = {}
    
    for location in locations:
        top_neighbours[location] = find_top_neighbours(location)
    top_neighbours_df = pd.DataFrame(top_neighbours).transpose()
    
    n[m[d]] = top_neighbours_df.loc["United Kingdom"]

In [77]:
eval_df = pd.DataFrame(eval_metric, index=[0])
eval_df

,euclidean,manhattan,chebyshev,cosine,cityblock,braycurtis,canberra,correlation,minkowski
0,0.683701,0.684481,0.686818,0.722468,0.684481,0.687987,0.698701,0.889935,0.683701


In [78]:
n_df = pd.DataFrame(n)
n_df

,euclidean,manhattan,chebyshev,cosine,cityblock,braycurtis,canberra,correlation,minkowski
0,Croatia,Croatia,Croatia,Croatia,Croatia,Croatia,Croatia,Croatia,Croatia
1,Ireland,Ireland,Ireland,Ireland,Ireland,Ireland,Netherlands,Ireland,Ireland
2,Slovenia,Slovenia,Slovenia,Slovenia,Slovenia,Slovenia,Ireland,Slovenia,Slovenia
3,Netherlands,Netherlands,Netherlands,Netherlands,Netherlands,Netherlands,Slovenia,Netherlands,Netherlands
4,France,Slovakia,Slovakia,Slovakia,Slovakia,Slovakia,Slovakia,Slovakia,Slovakia
